In [16]:
import pandas as pd
import json

# Function to load groups from an Excel file and create a dictionary
def load_groups_from_excel(file_path, sheet_name):
    # Read the Excel file
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    # Extract the relevant columns: 'ID', 'Name', 'Associated Groups'
    df_subset = df[['ID', 'name', 'associated groups']]
    # Create a dictionary for groups
    group_dict = {}
    for _, row in df_subset.iterrows():
        group_id = row['ID']
        group_name = row['name']
        associated_groups = row['associated groups']
        # Add the group information to the dictionary
        group_dict[group_id] = {
            'name': group_name,
            'associated_groups': associated_groups
        }
    return group_dict

# Step 1: Load the enterprise groups mapping from the local Excel file
enterprise_file_path = r"C:\Users\ricewater\Documents\CTITTP\ATTACK Excel sheets\enterprise-attack-v15.1-groups.xlsx"  
enterprise_groups = load_groups_from_excel(enterprise_file_path, sheet_name="groups")

# Step 2: Load the ICS groups mapping from the local Excel file
ics_file_path = r"C:\Users\ricewater\Documents\CTITTP\ATTACK Excel sheets\ics-attack-v15.1-groups.xlsx" 
ics_groups = load_groups_from_excel(ics_file_path, sheet_name="groups")

# Step 3: Load the Mobile groups mapping from the local Excel file
mobile_file_path = r"C:\Users\ricewater\Documents\CTITTP\ATTACK Excel sheets\mobile-attack-v15.1-groups.xlsx"
mobile_groups = load_groups_from_excel(mobile_file_path, sheet_name="groups")

# Step 4: Identify and add unique ICS and Mobile group IDs not in Enterprise
for group_id, group_info in ics_groups.items():
    if group_id not in enterprise_groups:
        # Add the unique ICS group to the enterprise groups mapping
        enterprise_groups[group_id] = group_info

for group_id, group_info in mobile_groups.items():
    if group_id not in enterprise_groups:
        # Add the unique Mobile group to the enterprise groups mapping
        enterprise_groups[group_id] = group_info

# Step 5: Optionally, export the updated group mapping to a new JSON file
with open('group_mapping_MITRE.json', 'w') as f:
    json.dump(enterprise_groups, f, indent=4)

# print("Updated group mapping exported to 'updated_group_mapping.json'.")

# Step 6: Display the counts and first few entries
print(f"\nTotal groups in updated mapping: {len(enterprise_groups)}")
print("\nSome example group mappings:")
for group_id, group_info in list(enterprise_groups.items())[:5]:  # Display first 5 for example
    print(f"ID: {group_id}, Name: {group_info['name']}, Associated Groups: {group_info['associated_groups']}")



Total groups in updated mapping: 152

Some example group mappings:
ID: G1028, Name: APT-C-23, Associated Groups: Arid Viper, Big Bang APT, Desert Falcon, Grey Karkadann, Mantis, TAG-63, Two-tailed Scorpion
ID: G0099, Name: APT-C-36, Associated Groups: Blind Eagle
ID: G0006, Name: APT1, Associated Groups: Comment Crew, Comment Group, Comment Panda
ID: G0005, Name: APT12, Associated Groups: DNSCALC, DynCalc, IXESHE, Numbered Panda
ID: G0023, Name: APT16, Associated Groups: nan


In [17]:
import json

# Function to read the actors data from the JSON file
def read_actors_from_file(filename="actors_data.json"):
    try:
        with open(filename, "r") as json_file:
            actors_data = json.load(json_file)  # Load the JSON data from the file
            return actors_data
    except FileNotFoundError:
        print(f"File {filename} not found.")
        return None
    except json.JSONDecodeError:
        print("Error decoding the JSON data.")
        return None

# Call the function to read the actors from the JSON file
malpedia_actors_data = read_actors_from_file()

In [19]:
# Compare MITRE group names and synonyms with Malpedia actors
def find_intersection_with_malpedia_actors(malpedia_actors_data, mitre_group_mappings):
    intersection = []  # Store MITRE groups that intersect with Malpedia actors
    unique_group_ids = set()  # To store unique group IDs for intersection
    mitre_groups_not_in_malpedia = []  # Store MITRE groups not found in Malpedia

    # Loop through each MITRE group in mitre_group_mappings
    for group_id, group_info in mitre_group_mappings.items():
        group_name = group_info['name'].lower()  # Normalize to lowercase for comparison
        associated_groups = group_info.get('associated_groups', '')

        # Normalize associated_groups to lowercase and ensure it's a list for iteration
        if isinstance(associated_groups, float) or associated_groups is None:
            associated_groups = ''
        associated_groups = [ag.lower() for ag in associated_groups] if isinstance(associated_groups, list) else [associated_groups.lower()]

        found_match = False

        # Loop through each actor in Malpedia actors data
        for actor_id, actor_info in malpedia_actors_data.items():
            actor_name = actor_info['value'].lower()  # Normalize actor name to lowercase
            synonyms = [synonym.lower() for synonym in actor_info.get('meta', {}).get('synonyms', [])]  # Normalize synonyms

            # Check if group_name or associated_groups match with Malpedia actor name or synonyms
            if group_name == actor_name or group_name in synonyms:
                intersection.append((group_name, actor_name, group_id))  # Found intersection with group_name
                unique_group_ids.add(group_id)
                found_match = True
                break  # No need to check further if match is found
            elif any(ag == actor_name or ag in synonyms for ag in associated_groups):
                intersection.append((group_name, actor_name, group_id))  # Found intersection with associated group
                unique_group_ids.add(group_id)
                found_match = True
                break

        # If no match was found for this MITRE group, add it to the "not in Malpedia" list
        if not found_match:
            mitre_groups_not_in_malpedia.append((group_name, group_id))

    return intersection, unique_group_ids, mitre_groups_not_in_malpedia


# Load the MITRE group mappings (this should be the mappings created from the Excel files)
group_mappings_file_path = 'group_mapping_MITRE.json'  # Replace with actual path to group mappings
with open(group_mappings_file_path, 'r') as f:
    mitre_group_mappings = json.load(f)

# Find the intersection
intersection, unique_group_ids, mitre_groups_not_in_malpedia = find_intersection_with_malpedia_actors(actors_data, mitre_group_mappings)

# Print the intersection results
print(f"Total intersection based on unique group IDs: {len(unique_group_ids)}")
if intersection:
    print("\nIntersection found between MITRE groups and Malpedia actors:")
    for group_name, actor_name, group_id in intersection:
        print(f"MITRE Group: {group_name} | Malpedia Actor: {actor_name} | Group ID: {group_id}")
else:
    print("No intersection found between MITRE groups and Malpedia actors.")

# Print the MITRE groups not present in Malpedia
if mitre_groups_not_in_malpedia:
    print(f"\nTotal MITRE groups not present in Malpedia: {len(mitre_groups_not_in_malpedia)}")
    for group_name, group_id in mitre_groups_not_in_malpedia:
        print(f"MITRE Group: {group_name} | Group ID: {group_id}")


Total intersection based on unique group IDs: 137

Intersection found between MITRE groups and Malpedia actors:
MITRE Group: apt-c-23 | Malpedia Actor: aridviper | Group ID: G1028
MITRE Group: apt-c-36 | Malpedia Actor: apt-c-36 | Group ID: G0099
MITRE Group: apt1 | Malpedia Actor: apt1 | Group ID: G0006
MITRE Group: apt12 | Malpedia Actor: apt12 | Group ID: G0005
MITRE Group: apt16 | Malpedia Actor: apt16 | Group ID: G0023
MITRE Group: apt17 | Malpedia Actor: apt17 | Group ID: G0025
MITRE Group: apt18 | Malpedia Actor: apt18 | Group ID: G0026
MITRE Group: apt19 | Malpedia Actor: apt19 | Group ID: G0073
MITRE Group: apt28 | Malpedia Actor: apt28 | Group ID: G0007
MITRE Group: apt29 | Malpedia Actor: apt29 | Group ID: G0016
MITRE Group: apt3 | Malpedia Actor: apt3 | Group ID: G0022
MITRE Group: apt30 | Malpedia Actor: apt30 | Group ID: G0013
MITRE Group: apt32 | Malpedia Actor: apt32 | Group ID: G0050
MITRE Group: apt33 | Malpedia Actor: apt33 | Group ID: G0064
MITRE Group: apt37 | Malp